In [1]:

import pandas as pd
from PIL import Image

from galaxy_mnist import GalaxyMNISTHighrez

/home/walml/miniforge3/envs/zoobot39_dev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset = GalaxyMNISTHighrez(
    root='/home/walml/repos/galaxy-datasets/data/external_data/galaxy_mnist',
    download=True,
    train=True  # by default, or set False for test set
)
test_dataset = GalaxyMNISTHighrez(
    root='/home/walml/repos/galaxy-datasets/data/external_data/galaxy_mnist',
    download=False,
    train=False  # by default, or set False for test set
)

Unpack back into images and catalog

In [3]:
train_images, train_labels = train_dataset.data.numpy().transpose(0, 2, 3, 1), train_dataset.targets
test_images, test_labels = test_dataset.data.numpy().transpose(0, 2, 3, 1), test_dataset.targets

In [4]:
train_images.shape

(8000, 224, 224, 3)

In [5]:
subfolder = '/home/walml/repos/galaxy-datasets/data/external_data/galaxy_mnist/images'

In [6]:
train_df = pd.DataFrame(data={'id_str': ['train_galaxy_' + str(n) for n in range(len(train_images))]})
train_df['label'] = train_labels
train_df['filename'] = train_df['id_str'] + '.jpg'
train_df['file_loc'] = subfolder + '/' + train_df['filename']

test_df = pd.DataFrame(data={'id_str': ['test_galaxy_' + str(n) for n in range(len(test_images))]})
test_df['label'] = test_labels
test_df['filename'] = test_df['id_str'] + '.jpg'
test_df['file_loc'] = subfolder + '/' + test_df['filename']

assert len(set(train_df['id_str']).intersection(set(test_df['id_str']))) == 0


In [7]:
train_images.shape

(8000, 224, 224, 3)

In [9]:
for galaxy_n, galaxy in train_df.iterrows():
    image = train_images[galaxy_n]
    Image.fromarray(image).save(galaxy['file_loc'])

for galaxy_n, galaxy in test_df.iterrows():
    image = test_images[galaxy_n]
    Image.fromarray(image).save(galaxy['file_loc'])

In [10]:
train_catalog_loc = '/home/walml/repos/galaxy-datasets/roots/galaxy_mnist/galaxy_mnist_train_catalog.parquet'
test_catalog_loc = '/home/walml/repos/galaxy-datasets/roots/galaxy_mnist/galaxy_mnist_test_catalog.parquet'

train_df.to_parquet(train_catalog_loc, index=False)
test_df.to_parquet(test_catalog_loc, index=False)

/home/walml/miniforge3/envs/zoobot39_dev/lib/python3.9/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [11]:
# tar czvf galaxy_mnist_images.tar.gz images

In [13]:
import hashlib

for loc in [
    train_catalog_loc,
    test_catalog_loc,
    '/home/walml/repos/galaxy-datasets/data/external_data/galaxy_mnist/galaxy_mnist_images.tar.gz'
]:
    # print hash
    with open(loc, 'rb') as f:
        md5_checksum = hashlib.md5(f.read()).hexdigest()

    print(md5_checksum)

cd22b21d165802f4bc1adf997424aec2
c5cca8d8afb6fb0d59baeb310a35d594
1c6cb0447f2f7ed676c3363ee194ced9
